### 핵심 동작 원리

첫 번째 카드: 항상 랜덤 선택
기억 검색: 첫 번째 카드와 같은 색상을 기억에서 찾기
두 번째 카드 선택:

기억에 있으면 → 그 위치로 직접 이동 <br>
기억에 없으면 → 랜덤 선택


메모리 관리: FIFO 방식으로 오래된 것부터 제거
매칭 성공시: 해당 카드들을 기억에서 제거

## 현재 게임 규칙

1단계2x2 / 총 그림 2개- 기본 카드: 2개- 트릭 카드: 0개제한 시간: 5초성공 리워드: 2원실패 리워드: 1원

2단계2x3 / 총 그림 3개- 기본 카드: 3개- 트릭 카드: 0개제한 시간: 7초성공 리워드: 2원

3단계3x4 / 총 그림 6개- 기본 카드: 6개- 트릭 카드: 0개제한 시간: 9초성공 리워드: 2원

4단계4x4 / 총 그림 8개- 기본 카드: 8개- 트릭 카드: 0개제한 시간: 13초성공 리워드: 2원

5단계4x4 / 총 그림 8개- 기본 카드: 6개- 트릭 카드: 2개제한 시간: 15초성공 리워드: 3원

6단계4x5 / 총 그림 10개- 기본 카드: 10개- 트릭 카드: 0개제한 시간: 21초성공 리워드: 3원

7단계4x5 / 총 그림 10개- 기본 카드: 8개- 트릭 카드: 2개제한 시간: 25초성공 리워드: 6원

- 재참여 : 최대 3번까지 플레이 가능. 이 경우, 1단계부터 재시작하며 광고를 시청한다. 매출: 1원
- 재시도 : 무재한 재시도 가능. 이 경우, 기존 실패한 단계에서 재시작하며 광고를 시청 한다. 매출 : 1원

In [1]:
import random
import numpy as np
from collections import deque, Counter
from typing import List, Tuple, Dict
import pandas as pd

class Card:
    """카드 클래스"""
    def __init__(self, color: int, position: int):
        self.color = color
        self.position = position
        self.is_matched = False

class LimitedMemoryPlayer:
    """최근 N개 카드만 기억하는 플레이어 클래스"""
    def __init__(self, memory_size: int = 3):
        self.memory_size = memory_size
        self.memory = {}  # position -> color 매핑
        self.memory_order = deque(maxlen=memory_size)  # FIFO 순서 관리
        
    def remember_card(self, position: int, color: int):
        """카드를 기억에 추가"""
        if position in self.memory:
            self.memory_order.remove(position)
            self.memory_order.append(position)
        else:
            if len(self.memory) >= self.memory_size:
                oldest_pos = self.memory_order.popleft()
                del self.memory[oldest_pos]
            
            self.memory[position] = color
            self.memory_order.append(position)
    
    def find_matching_card(self, target_color: int, exclude_positions: List[int] = None) -> int:
        """기억에서 같은 색상의 카드 위치 찾기"""
        if exclude_positions is None:
            exclude_positions = []
            
        for position, color in self.memory.items():
            if color == target_color and position not in exclude_positions:
                return position
        return -1
    
    def remove_from_memory(self, positions: List[int]):
        """매칭된 카드들을 기억에서 제거"""
        for pos in positions:
            if pos in self.memory:
                del self.memory[pos]
                if pos in self.memory_order:
                    self.memory_order.remove(pos)

def create_deck(n_pairs: int) -> List[Card]:
    """n개 쌍의 카드 덱 생성"""
    deck = []
    colors = list(range(n_pairs)) * 2
    random.shuffle(colors)
    
    for i, color in enumerate(colors):
        deck.append(Card(color, i))
    
    return deck

def get_available_positions(deck: List[Card]) -> List[int]:
    """아직 매칭되지 않은 카드 위치들 반환"""
    return [i for i, card in enumerate(deck) if not card.is_matched]

def simulate_single_game(n_pairs: int, memory_size: int) -> Tuple[int, bool]:
    """단일 게임 시뮬레이션"""
    deck = create_deck(n_pairs)
    player = LimitedMemoryPlayer(memory_size)
    turns = 0
    matched_pairs = 0
    
    while matched_pairs < n_pairs:
        turns += 1
        
        # 사용 가능한 카드들
        available_positions = get_available_positions(deck)
        
        if len(available_positions) < 2:
            break
        
        # 첫 번째 카드 선택 (랜덤)
        first_pos = random.choice(available_positions)
        first_card = deck[first_pos]
        
        # 첫 번째 카드를 기억에 추가
        player.remember_card(first_pos, first_card.color)
        
        # 기억에서 같은 색상의 카드 찾기
        matching_pos = player.find_matching_card(first_card.color, [first_pos])
        
        if matching_pos != -1 and matching_pos in available_positions:
            # 기억에서 매칭 카드를 찾은 경우
            second_pos = matching_pos
            second_card = deck[second_pos]
            
            # 매칭 성공
            first_card.is_matched = True
            second_card.is_matched = True
            matched_pairs += 1
            
            # 매칭된 카드들을 기억에서 제거
            player.remove_from_memory([first_pos, second_pos])
        else:
            # 기억에 없으므로 랜덤하게 두 번째 카드 선택
            available_for_second = [pos for pos in available_positions if pos != first_pos]
            
            if not available_for_second:
                break
                
            second_pos = random.choice(available_for_second)
            second_card = deck[second_pos]
            
            # 두 번째 카드를 기억에 추가
            player.remember_card(second_pos, second_card.color)
            
            if first_card.color == second_card.color:
                # 운 좋게 매칭
                first_card.is_matched = True
                second_card.is_matched = True
                matched_pairs += 1
                
                # 매칭된 카드들을 기억에서 제거
                player.remove_from_memory([first_pos, second_pos])
    
    return turns, matched_pairs == n_pairs

def calculate_turn_probability_density(turns_list: List[int], max_turns: int) -> Dict[int, float]:
    """각 턴당 확률 밀도 계산"""
    turn_counter = Counter(turns_list)
    total_games = len(turns_list)
    
    probability_density = {}
    for turn in range(1, max_turns + 1):
        count = turn_counter.get(turn, 0)
        probability_density[f"turn_{turn}"] = count / total_games
    
    return probability_density

def run_detailed_simulation(n_pairs: int, memory_size: int, num_simulations: int = 10000) -> Dict:
    """상세 결과를 포함한 시뮬레이션 실행"""
    turns_results = []
    successful_games = 0
    
    # 시뮬레이션 실행
    for _ in range(num_simulations):
        turns, success = simulate_single_game(n_pairs, memory_size)
        turns_results.append(turns)
        if success:
            successful_games += 1
    
    # 기본 통계 계산
    min_turns = min(turns_results)
    max_turns = max(turns_results)
    average_turns = np.mean(turns_results)
    std_turns = np.std(turns_results)
    success_rate = successful_games / num_simulations
    
    # 확률 밀도 계산
    probability_density = calculate_turn_probability_density(turns_results, max_turns)
    
    # 결과 딕셔너리 구성
    result = {
        'n_pairs': n_pairs,
        'memory_size': memory_size,
        'num_simulations': num_simulations,
        'min_turns': min_turns,
        'max_turns': max_turns,
        'average_turns': round(average_turns, 3),
        'std_turns': round(std_turns, 3),
        'success_rate': round(success_rate, 4),
        **probability_density  # turn_1, turn_2, ... 확률 밀도 추가
    }
    
    return result

def run_batch_detailed_simulation(n_pairs_list: List[int], 
                                memory_sizes: List[int], 
                                num_simulations: int = 10000) -> List[Dict]:
    """여러 조건에 대한 배치 시뮬레이션"""
    results = []
    
    for n_pairs in n_pairs_list:
        for memory_size in memory_sizes:
            print(f"시뮬레이션 중: {n_pairs}쌍, 메모리 크기 {memory_size}...")
            result = run_detailed_simulation(n_pairs, memory_size, num_simulations)
            results.append(result)
    
    return results

def normalize_probability_densities(results: List[Dict]) -> List[Dict]:
    """확률 밀도를 가장 큰 max_turns에 맞춰 정규화"""
    # 모든 결과에서 최대 턴 수 찾기
    global_max_turns = max(result['max_turns'] for result in results)
    
    normalized_results = []
    for result in results:
        normalized_result = result.copy()
        
        # 부족한 턴에 대해 0.0으로 채우기
        for turn in range(1, global_max_turns + 1):
            turn_key = f"turn_{turn}"
            if turn_key not in normalized_result:
                normalized_result[turn_key] = 0.0
        
        normalized_results.append(normalized_result)
    
    return normalized_results

def results_to_dataframe(results: List[Dict]) -> pd.DataFrame:
    """결과를 DataFrame으로 변환"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # 열 순서 정렬 (기본 정보 + turn_1, turn_2, ...)
    basic_cols = ['n_pairs', 'memory_size', 'num_simulations', 'min_turns', 
                  'max_turns', 'average_turns', 'std_turns', 'success_rate']
    
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))  # turn_1, turn_2, ... 순서로 정렬
    
    column_order = basic_cols + turn_cols
    df = df[column_order]
    
    return df

def print_summary_statistics(results: List[Dict]):
    """요약 통계 출력"""
    print("\n=== 시뮬레이션 결과 요약 ===")
    print("=" * 80)
    
    for result in results:
        n_pairs = result['n_pairs']
        memory_size = result['memory_size']
        avg_turns = result['average_turns']
        std_turns = result['std_turns']
        min_turns = result['min_turns']
        max_turns = result['max_turns']
        success_rate = result['success_rate']
        
        print(f"\n【{n_pairs}쌍, 메모리 {memory_size}개】")
        print(f"  평균 턴 수: {avg_turns:.3f} ± {std_turns:.3f}")
        print(f"  범위: {min_turns} ~ {max_turns} 턴")
        print(f"  성공률: {success_rate:.1%}")
        
        # 확률이 높은 상위 3개 턴 출력
        turn_probs = [(k, v) for k, v in result.items() if k.startswith('turn_') and v > 0]
        turn_probs.sort(key=lambda x: x[1], reverse=True)
        
        print("  가장 빈번한 턴 수:")
        for turn_key, prob in turn_probs[:3]:
            turn_num = turn_key.split('_')[1]
            print(f"    {turn_num}턴: {prob:.1%}")
def extract_probability_density_dataframe(results: List[Dict]) -> pd.DataFrame:
    """인덱스: (n_pairs, memory_size), 컬럼: turn_1 ~ turn_n 형태의 DataFrame 추출"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # turn 컬럼들만 추출 및 정렬
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    # 필요한 컬럼들만 선택
    probability_df = df[['n_pairs', 'memory_size'] + turn_cols].copy()
    
    # 인덱스 설정
    probability_df.set_index(['n_pairs', 'memory_size'], inplace=True)
    
    return probability_df
# 실행 예제
if __name__ == "__main__":
    print("=== 상세 결과 카드 게임 시뮬레이션 ===\n")
    
    # 시뮬레이션 설정
    n_pairs_list = [2, 3, 4]  # 카드 쌍 수
    memory_sizes = [1, 2, 3, 5]  # 메모리 크기
    num_simulations = 5000  # 시뮬레이션 횟수
    
    # 배치 시뮬레이션 실행
    results = run_batch_detailed_simulation(n_pairs_list, memory_sizes, num_simulations)
    
    # 요약 통계 출력
    print_summary_statistics(results)
    
    # DataFrame으로 변환
    df = results_to_dataframe(results)
    
    print(f"\n=== DataFrame 형태 결과 ===")
    print("형태:", df.shape)
    print("\n기본 정보 열:")
    basic_cols = ['n_pairs', 'memory_size', 'average_turns', 'min_turns', 'max_turns', 'success_rate']
    print(df[basic_cols].to_string(index=False))
    
    print(f"\n=== 확률 밀도 예시 (첫 번째 결과) ===")
    first_result = results[0]
    print(f"{first_result['n_pairs']}쌍, 메모리 {first_result['memory_size']}개:")
    
    turn_cols = [k for k in first_result.keys() if k.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    for turn_key in turn_cols[:10]:  # 처음 10턴만 출력
        turn_num = turn_key.split('_')[1]
        prob = first_result[turn_key]
        if prob > 0:
            print(f"  {turn_num}턴: {prob:.4f} ({prob:.1%})")
    
    # 이론적 완벽 플레이어와 비교
    print(f"\n=== 이론적 완벽 플레이어 비교 ===")
    for n_pairs in n_pairs_list:
        theoretical_perfect = 1.61 * n_pairs
        print(f"\n{n_pairs}쌍:")
        print(f"  이론적 완벽 플레이어: {theoretical_perfect:.2f}턴")
        
        # 해당 쌍에 대한 결과들 출력
        pair_results = [r for r in results if r['n_pairs'] == n_pairs]
        for result in pair_results:
            memory_size = result['memory_size']
            avg_turns = result['average_turns']
            diff = avg_turns - theoretical_perfect
            print(f"  메모리 {memory_size}개: {avg_turns:.2f}턴 (차이: {diff:+.2f})")
    
    # 전체 결과 반환을 위한 함수도 제공
    def get_simulation_results():
        """시뮬레이션 결과 반환 (다른 스크립트에서 사용 가능)"""
        return results, df, prob_df

=== 상세 결과 카드 게임 시뮬레이션 ===

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 4쌍, 메모리 크기 1...
시뮬레이션 중: 4쌍, 메모리 크기 2...
시뮬레이션 중: 4쌍, 메모리 크기 3...
시뮬레이션 중: 4쌍, 메모리 크기 5...

=== 시뮬레이션 결과 요약 ===

【2쌍, 메모리 1개】
  평균 턴 수: 4.032 ± 2.564
  범위: 2 ~ 23 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 33.5%
    3턴: 22.4%
    4턴: 14.6%

【2쌍, 메모리 2개】
  평균 턴 수: 3.330 ± 1.436
  범위: 2 ~ 16 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 33.8%
    2턴: 32.4%
    4턴: 17.4%

【2쌍, 메모리 3개】
  평균 턴 수: 2.960 ± 0.889
  범위: 2 ~ 8 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 45.3%
    2턴: 32.7%
    4턴: 16.9%

【2쌍, 메모리 5개】
  평균 턴 수: 2.952 ± 0.890
  범위: 2 ~ 8 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 45.1%
    2턴: 33.3%
    4턴: 16.3%

【3쌍, 메모리 1개】
  평균 턴 수: 8.942 ± 5.043
  범위: 3 ~ 43 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    5턴: 11.5%
    6턴: 11.0%
    4턴: 9.9%

【3쌍, 메모리 2개】
  평균 턴 수: 6.747 ± 2.891
  

In [2]:
results = run_batch_detailed_simulation([2, 3, 6,8,10], [1, 2, 3, 4,5,6,7,8,9,10,11,12], 20000)

# 요청하신 형태의 DataFrame 추출
prob_df = extract_probability_density_dataframe(results)
prob_df

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 4...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 2쌍, 메모리 크기 6...
시뮬레이션 중: 2쌍, 메모리 크기 7...
시뮬레이션 중: 2쌍, 메모리 크기 8...
시뮬레이션 중: 2쌍, 메모리 크기 9...
시뮬레이션 중: 2쌍, 메모리 크기 10...
시뮬레이션 중: 2쌍, 메모리 크기 11...
시뮬레이션 중: 2쌍, 메모리 크기 12...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 4...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 6...
시뮬레이션 중: 3쌍, 메모리 크기 7...
시뮬레이션 중: 3쌍, 메모리 크기 8...
시뮬레이션 중: 3쌍, 메모리 크기 9...
시뮬레이션 중: 3쌍, 메모리 크기 10...
시뮬레이션 중: 3쌍, 메모리 크기 11...
시뮬레이션 중: 3쌍, 메모리 크기 12...
시뮬레이션 중: 6쌍, 메모리 크기 1...
시뮬레이션 중: 6쌍, 메모리 크기 2...
시뮬레이션 중: 6쌍, 메모리 크기 3...
시뮬레이션 중: 6쌍, 메모리 크기 4...
시뮬레이션 중: 6쌍, 메모리 크기 5...
시뮬레이션 중: 6쌍, 메모리 크기 6...
시뮬레이션 중: 6쌍, 메모리 크기 7...
시뮬레이션 중: 6쌍, 메모리 크기 8...
시뮬레이션 중: 6쌍, 메모리 크기 9...
시뮬레이션 중: 6쌍, 메모리 크기 10...
시뮬레이션 중: 6쌍, 메모리 크기 11...
시뮬레이션 중: 6쌍, 메모리 크기 12...
시뮬레이션 중: 8쌍, 메모리 크기 1...
시뮬레이션 중: 8쌍, 메모리 크기 2...
시뮬레이션 중: 8쌍, 메모리 크기 3...
시뮬레이션 중: 8쌍, 메모리

turn_1   turn_2   turn_3   turn_4   turn_5   turn_6  \
n_pairs memory_size                                                        
2       1               0.0  0.33780  0.21915  0.14555  0.09985  0.06635   
        2               0.0  0.33715  0.33155  0.16605  0.07835  0.04340   
        3               0.0  0.33045  0.44540  0.17115  0.04120  0.00860   
        4               0.0  0.33775  0.44370  0.16180  0.04265  0.01090   
        5               0.0  0.33575  0.44225  0.16710  0.04215  0.01025   
        6               0.0  0.32990  0.44725  0.16725  0.04335  0.00995   
        7               0.0  0.33730  0.44675  0.16185  0.04245  0.00915   
        8               0.0  0.33595  0.44445  0.16650  0.04110  0.00935   
        9               0.0  0.33560  0.44240  0.16705  0.04210  0.01030   
        10              0.0  0.32995  0.44500  0.16960  0.04360  0.00910   
        11              0.0  0.33795  0.44205  0.16540  0.04275  0.00935   
        12              0.0  0.33355  0.44005  0.16840  0.04425  0.01025   
3       1               0.0  0.00000  0.06700  0.09880  0.10570  0.10450   
        2               0.0  0.00000  0.06820  0.15775  0.18375  0.15990   
        3               0.0  0.00000  0.06865  0.27215  0.27135  0.17840   
        4               0.0  0.00000  0.06625  0.28845  0.32865  0.18205   
        5               0.0  0.00000  0.06385  0.28655  0.33940  0.19180   
        6               0.0  0.00000  0.06880  0.27875  0.33550  0.19755   
        7               0.0  0.00000  0.06715  0.28695  0.33955  0.19055   
        8               0.0  0.00000  0.06695  0.27860  0.33980  0.19545   
        9               0.0  0.00000  0.06220  0.28235  0.35060  0.18945   
        10              0.0  0.00000  0.06715  0.28515  0.33870  0.19160   
        11              0.0  0.00000  0.06525  0.28885  0.33355  0.19345   
        12              0.0  0.00000  0.06555  0.28420  0.33905  0.19165   
6       1               0.0  0.00000  0.00000  0.00000  0.00000  0.00015   
        2               0.0  0.00000  0.00000  0.00000  0.00000  0.00015   
        3               0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        4               0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        5               0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        6               0.0  0.00000  0.00000  0.00000  0.00000  0.00015   
        7               0.0  0.00000  0.00000  0.00000  0.00000  0.00020   
        8               0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        9               0.0  0.00000  0.00000  0.00000  0.00000  0.00010   
        10              0.0  0.00000  0.00000  0.00000  0.00000  0.00010   
        11              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        12              0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
8       1               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        2               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        3               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        4               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        5               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        6               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        7               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        8               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        9               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        10              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        11              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        12              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
10      1               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        2               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        3               0.0  0.00000  0.00000  0.00000  0.00000  0.00

In [3]:
import pandas as pd
import numpy as np
import random
from scipy import stats
from typing import Dict, List, Tuple
from collections import defaultdict

class GameLevel:
    """게임 레벨 정보"""
    def __init__(self, level: int, n_pairs: int, time_limit: float, 
                 success_reward: int, fail_reward: int = 1, 
                 turn_duration: float = 0.88, trick_cards: int = 0):
        self.level = level
        self.n_pairs = n_pairs
        self.time_limit = time_limit
        self.success_reward = success_reward
        self.fail_reward = fail_reward
        self.turn_duration = turn_duration
        self.trick_cards = trick_cards  # 트릭카드 개수
        self.basic_cards = n_pairs - trick_cards  # 기본카드 개수
        
        # 트릭카드로 인한 체감 난이도 증가를 turn_duration에 반영
        if trick_cards > 0:
            # 트릭카드 1개당 20% 속도 감소 (더 신중하게 플레이)
            self.effective_turn_duration = turn_duration * (1 + trick_cards * 0.1)
        elif level < 4:
            self.effective_turn_duration = turn_duration * (0.6)
        else:
            self.effective_turn_duration = turn_duration

class GameSimulator:
    """완전한 게임 시뮬레이션 엔진 - 현실적 재시도 모델 적용"""
    
    def __init__(self, 
                 probability_df: pd.DataFrame,
                 game_levels: Dict = None,
                 user_memory_distribution: Dict = None,
                 max_retries: int = 8,
                 retry_model: str = 'realistic',  # 'uniform' 또는 'realistic'
                 turn_time_params: Dict = None):
        """
        초기화
        
        Args:
            probability_df: extract_probability_density_dataframe()으로 생성된 확률 분포 DataFrame
            game_levels: 게임 레벨 설정 (None이면 기본값 사용)
            user_memory_distribution: 유저 메모리 분포 (None이면 기본값 사용)
            max_retries: 최대 재시도 횟수
            retry_model: 'uniform' (기존 0.5 고정) 또는 'realistic' (동적 계산)
            turn_time_params: 턴 시간 파라미터
        """
        self.prob_df = probability_df
        self.game_levels = game_levels or self._get_default_game_levels()
        self.user_memory_distribution = user_memory_distribution or self._get_default_memory_distribution()
        self.max_retries = max_retries
        self.retry_model = retry_model
        self.turn_time_params = turn_time_params or self._get_default_turn_time_params()
        
        # 현실적 재시도 모델 설정
        if retry_model == 'realistic':
            self.user_types = {
                'casual': {'base_rate': 0.25, 'distribution_prob': 0.40},
                'regular': {'base_rate': 0.50, 'distribution_prob': 0.45}, 
                'hardcore': {'base_rate': 0.75, 'distribution_prob': 0.15}
            }
        else:
            # uniform 모델을 위한 고정 재시도율
            self.uniform_retry_probability = 0.5
    
    def _get_default_game_levels(self) -> Dict:
        """기본 게임 레벨 설정"""
        return {
            1: GameLevel(1, 2, 5.0, 2, 1, 0.8, 0),    # 트릭카드 없음
            2: GameLevel(2, 3, 7.0, 2, 1, 0.9, 0),    # 트릭카드 없음
            3: GameLevel(3, 6, 9.0, 2, 1, 1.0, 0),    # 트릭카드 없음
            4: GameLevel(4, 8, 13.0, 2, 1, 1.1, 0),   # 트릭카드 없음
            5: GameLevel(5, 8, 15.0, 3, 1, 1.2, 2),   # 트릭카드 2개 → 1.2 * 1.4 = 1.68초
            6: GameLevel(6, 10, 21.0, 3, 1, 1.3, 0),  # 트릭카드 없음
            7: GameLevel(7, 10, 25.0, 6, 1, 1.4, 2)   # 트릭카드 2개 → 1.4 * 1.4 = 1.96초
        }
    
    def _get_default_memory_distribution(self) -> Dict:
        """기본 유저 메모리 분포"""
        return {
            1: 0.02,
            2: 0.05,
            3: 0.08,
            4: 0.15,
            5: 0.245,   # 중앙(최대)
            6: 0.19,
            7: 0.13,
            8: 0.075,
            9: 0.04,
            10: 0.02
        }

    def _get_default_turn_time_params(self) -> Dict:
        """기본 턴 시간 파라미터"""
        return {
            'distribution': 't',
            'df': 10,
            'loc': 0.88,
            'scale': 1,
            'min_time': 0.1
        }
    
    def sample_user_profile(self) -> Dict:
        """유저 프로필 생성 (메모리 + 재시도 성향)"""
        memory_size = self.sample_user_memory()
        
        if self.retry_model == 'realistic':
            # 유저 타입 결정
            types = list(self.user_types.keys())
            probs = [self.user_types[t]['distribution_prob'] for t in types]
            user_type = np.random.choice(types, p=probs)
            
            return {
                'memory_size': memory_size,
                'user_type': user_type,
                'base_retry_rate': self.user_types[user_type]['base_rate'],
                'individual_factor': np.random.uniform(0.8, 1.2),  # 개인차 ±20%
                'patience_level': np.random.uniform(0.5, 1.5)     # 인내심 레벨
            }
        else:
            return {
                'memory_size': memory_size,
                'user_type': 'uniform',
                'base_retry_rate': self.uniform_retry_probability,
                'individual_factor': 1.0,
                'patience_level': 1.0
            }
    
    def sample_user_memory(self) -> int:
        """유저 메모리 크기 샘플링"""
        memory_sizes = list(self.user_memory_distribution.keys())
        probabilities = list(self.user_memory_distribution.values())
        return np.random.choice(memory_sizes, p=probabilities)
    
    def calculate_dynamic_retry_probability(self, 
                                          user_profile: Dict,
                                          level: int,
                                          consecutive_failures: int,
                                          total_attempts: int) -> float:
        """동적 재시도 확률 계산"""
        
        if self.retry_model == 'uniform':
            return self.uniform_retry_probability  # 기존 방식
        
        # 현실적 모델
        game_level = self.game_levels[level]
        base_rate = user_profile['base_retry_rate']
        
        # 1. 보상 크기 영향 (높은 보상일수록 재시도율 증가)
        reward_factor = 1.0 + (game_level.success_reward - 2) * 0.15
        
        # 2. 연속 실패 페널티 (기하급수적 감소)
        failure_decay = 0.80 ** consecutive_failures
        
        # 3. 레벨 진행도 보너스 (후반부일수록 포기하기 아까움)
        progress_bonus = 1.0 + (level / len(self.game_levels)) * 0.25
        
        # 4. 피로도 (전체 시도 횟수가 많을수록 감소)
        fatigue_factor = 0.95 ** (total_attempts // 8)
        
        # 5. 트릭카드 스트레스 (트릭카드가 있으면 재시도 의욕 감소)
        trick_stress = 0.9 if game_level.trick_cards > 0 else 1.0
        
        # 6. 개인차 및 인내심
        personal_factor = user_profile['individual_factor'] * user_profile['patience_level']
        
        # 최종 확률 계산
        final_prob = (base_rate * 
                      reward_factor * 
                      failure_decay * 
                      progress_bonus * 
                      fatigue_factor * 
                      trick_stress * 
                      personal_factor)
        
        # 0-1 범위로 제한
        return np.clip(final_prob, 0.0, 0.95)  # 최대 95%로 제한
    
    def sample_turn_count(self, n_pairs: int, memory_size: int) -> int:
        """확률 분포표에서 턴 수 샘플링"""
        try:
            # MultiIndex 확인
            if isinstance(self.prob_df.index, pd.MultiIndex):
                # MultiIndex인 경우 (n_pairs, memory_size)
                available_memories = self.prob_df.index.get_level_values(1).unique()
                if memory_size not in available_memories:
                    memory_size = min(available_memories, key=lambda x: abs(x - memory_size))
                
                prob_row = self.prob_df.loc[(n_pairs, memory_size)]
            else:
                # 단일 인덱스인 경우 - memory_size만 사용
                if memory_size not in self.prob_df.index:
                    available_memories = self.prob_df.index.unique()
                    memory_size = min(available_memories, key=lambda x: abs(x - memory_size))
                
                prob_row = self.prob_df.loc[memory_size]
            
            # 0이 아닌 확률들만 추출
            non_zero_probs = prob_row[prob_row > 0]
            
            if len(non_zero_probs) == 0:
                # 확률이 모두 0인 경우 기본값 반환
                return n_pairs
            
            # 확률 정규화
            probabilities = non_zero_probs.values
            probabilities = probabilities / np.sum(probabilities)
            
            # 턴 번호 추출 (컬럼명에서 숫자 부분만 추출)
            turn_numbers = []
            for col in non_zero_probs.index:
                if str(col).startswith('turn_'):
                    turn_numbers.append(int(col.split('_')[1]))
                else:
                    turn_numbers.append(int(col))
            
            # 샘플링
            sampled_turn = np.random.choice(turn_numbers, p=probabilities)
            return sampled_turn
            
        except (KeyError, IndexError) as e:
            print(f"경고: ({n_pairs}쌍, 메모리{memory_size}) 조건이 없어서 기본값 {n_pairs} 사용. 에러: {e}")
            return n_pairs
    
    def sample_turn_time(self, turn_count: int, game_level: GameLevel = None) -> float:
        """턴당 소요 시간 샘플링 (트릭카드 영향 반영)"""
        params = self.turn_time_params
        
        # 기본 턴 지속시간
        base_turn_duration = params['loc']
        
        # 트릭카드로 인한 체감 난이도 반영
        if game_level and hasattr(game_level, 'trick_cards') and game_level.trick_cards > 0:
            # 트릭카드 1개당 20% 속도 감소 (더 신중하게 플레이)
            difficulty_multiplier = 1 + (game_level.trick_cards * 0.2)
            effective_turn_duration = base_turn_duration * difficulty_multiplier
        else:
            effective_turn_duration = base_turn_duration
        
        # 분포에 따른 샘플링
        if params['distribution'] == 't':
            # T분포 샘플링
            turn_times = stats.t.rvs(
                df=params['df'], 
                loc=effective_turn_duration,
                scale=params['scale'], 
                size=turn_count
            )
        elif params['distribution'] == 'normal':
            # 정규분포 샘플링
            turn_times = np.random.normal(
                loc=effective_turn_duration,
                scale=params['scale'], 
                size=turn_count
            )
        elif params['distribution'] == 'uniform':
            # 균등분포 샘플링
            turn_times = np.random.uniform(
                low=effective_turn_duration - params['scale'],
                high=effective_turn_duration + params['scale'],
                size=turn_count
            )
        else:
            # 기본값: 고정 시간
            turn_times = np.full(turn_count, effective_turn_duration)
        
        # 음수 시간 제거 (최소 시간 적용)
        turn_times = np.maximum(turn_times, params['min_time'])
        
        return np.sum(turn_times)
    
    def simulate_level(self, level: int, user_profile: Dict, 
                      total_attempts_so_far: int = 0) -> Dict:
        """단일 레벨 시뮬레이션 - 동적 재시도 확률 적용"""
        game_level = self.game_levels[level]
        attempts = 0
        successes = 0      # 성공한 시도 수
        failures = 0       # 실패한 시도 수
        retries = 0        # 재시도한 횟수 (광고 시청)
        abandons = 0       # 이탈한 횟수 (광고 시청 안함)
        total_cost = 0     # 우리가 유저에게 지출한 게임 리워드 비용
        total_revenue = 0  # 유저가 재시도하여 광고 시청 매출
        consecutive_failures = 0
        
        for attempt in range(self.max_retries + 1):  # 최초 시도 + 재시도
            attempts += 1
            
            # 턴 수 샘플링
            turn_count = self.sample_turn_count(game_level.n_pairs, user_profile['memory_size'])
            
            # 소요 시간 샘플링 (트릭카드 영향 반영)
            total_time = self.sample_turn_time(turn_count, game_level)
            
            # 성공/실패 판정
            if total_time <= game_level.time_limit:
                # 성공 - 유저에게 리워드 지급 (우리 비용)
                successes += 1
                total_cost = game_level.success_reward
                break  # 성공하면 레벨 완료
            else:
                # 실패
                failures += 1
                consecutive_failures += 1
                
                # 마지막 시도가 아닌 경우 재시도 여부 결정
                if attempt < self.max_retries:
                    # 동적 재시도 확률 계산
                    retry_prob = self.calculate_dynamic_retry_probability(
                        user_profile, level, consecutive_failures, 
                        total_attempts_so_far + attempts
                    )
                    
                    if np.random.random() < retry_prob:
                        # 재시도 = 광고 시청 = 매출 발생
                        retries += 1
                        total_revenue += game_level.fail_reward
                        continue
                    else:
                        # 포기 = 이탈
                        abandons += 1
                        break
                else:
                    # 마지막 시도 실패 = 자동 이탈
                    abandons += 1
                    break
        
        level_success = successes > 0  # 레벨 완료 여부
        
        return {
            'level': level,
            'memory_size': user_profile['memory_size'],
            'user_type': user_profile['user_type'],
            'attempts': attempts,
            'successes': successes,
            'failures': failures,
            'retries': retries,        # 재시도 횟수 (매출 발생)
            'abandons': abandons,      # 이탈 횟수 (매출 없음)
            'level_success': level_success,
            'success': level_success,
            'total_cost': total_cost,
            'total_revenue': total_revenue,
            'net_profit': total_revenue - total_cost,
            'n_pairs': game_level.n_pairs,
            'time_limit': game_level.time_limit,
            'turn_duration': game_level.turn_duration,
            'trick_cards': game_level.trick_cards,
            'consecutive_failures': consecutive_failures
        }
    
    def simulate_full_game(self, user_profile: Dict) -> Dict:
        """전체 게임 시뮬레이션 - 동적 재시도 확률 적용"""
        results = []
        current_level = 1
        total_attempts_count = 0
        
        total_stats = {
            'total_attempts': 0,
            'total_successes': 0,  # 성공한 시도 수
            'total_failures': 0,   # 실패한 시도 수
            'total_retries': 0,    # 재시도 횟수 (매출 발생)
            'total_abandons': 0,   # 이탈 횟수 (매출 없음)
            'total_cost': 0,
            'total_revenue': 0,
            'levels_completed': 0,
            'final_level': 0
        }
        
        while current_level <= len(self.game_levels):
            level_result = self.simulate_level(current_level, user_profile, total_attempts_count)
            results.append(level_result)
            
            # 통계 업데이트
            total_attempts_count += level_result['attempts']
            total_stats['total_attempts'] += level_result['attempts']
            total_stats['total_successes'] += level_result['successes']
            total_stats['total_failures'] += level_result['failures']
            total_stats['total_retries'] += level_result['retries']
            total_stats['total_abandons'] += level_result['abandons']
            total_stats['total_cost'] += level_result['total_cost']
            total_stats['total_revenue'] += level_result['total_revenue']
            total_stats['final_level'] = current_level
            
            if level_result['level_success']:  # 레벨 완료 여부 확인
                total_stats['levels_completed'] += 1
                current_level += 1
            else:
                break  # 레벨 실패로 게임 종료
        
        total_stats['net_profit'] = total_stats['total_revenue'] - total_stats['total_cost']
        
        return {
            'user_profile': user_profile,
            'level_results': results,
            'total_stats': total_stats
        }
    
    def run_simulation(self, num_users: int = 10000) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """대규모 시뮬레이션 실행"""
        print(f"=== {num_users}명 유저 게임 시뮬레이션 시작 (재시도 모델: {self.retry_model}) ===")
        
        user_results = []
        level_stats = defaultdict(lambda: {
            'attempts': 0, 'successes': 0, 'failures': 0, 
            'retries': 0, 'abandons': 0,  # 재시도와 이탈 구분
            'total_cost': 0, 'total_revenue': 0
        })
        user_type_stats = defaultdict(int)  # 유저 타입 통계
        
        for user_id in range(num_users):
            if (user_id + 1) % 1000 == 0:
                print(f"진행률: {user_id + 1}/{num_users} ({(user_id + 1)/num_users*100:.1f}%)")
            
            user_profile = self.sample_user_profile()
            user_type_stats[user_profile['user_type']] += 1
            game_result = self.simulate_full_game(user_profile)
            user_results.append(game_result)
            
            for level_result in game_result['level_results']:
                level = level_result['level']
                
                # 시도 단위로 누적 (재시도와 이탈 구분)
                level_stats[level]['attempts'] += level_result['attempts']
                level_stats[level]['successes'] += level_result['successes']
                level_stats[level]['failures'] += level_result['failures']
                level_stats[level]['retries'] += level_result['retries']      # 재시도 횟수
                level_stats[level]['abandons'] += level_result['abandons']    # 이탈 횟수
                level_stats[level]['total_cost'] += level_result['total_cost']
                level_stats[level]['total_revenue'] += level_result['total_revenue']
        
        # 유저 타입 분포 출력
        if self.retry_model == 'realistic':
            print(f"\n📊 유저 타입 분포:")
            for user_type, count in user_type_stats.items():
                print(f"  {user_type}: {count}명 ({count/num_users*100:.1f}%)")
        
        user_df = self._create_user_dataframe(user_results)
        level_df = self._create_level_dataframe(level_stats)
        
        return user_df, level_df
    
    def _create_user_dataframe(self, user_results: List[Dict]) -> pd.DataFrame:
        """유저 결과 DataFrame 생성 - 유저 타입 정보 추가"""
        rows = []
        for i, result in enumerate(user_results):
            total_stats = result['total_stats']
            user_profile = result['user_profile']
            
            # 재시도 횟수 계산 (실패한 시도 횟수)
            retry_count = total_stats['total_failures']
            
            row = {
                'user_id': i + 1,
                'memory_size': user_profile['memory_size'],
                'user_type': user_profile['user_type'],
                'base_retry_rate': user_profile['base_retry_rate'],
                **total_stats,
                'retry_count': retry_count  # 재시도 횟수 추가
            }
            rows.append(row)
        
        return pd.DataFrame(rows)
        
    def _create_level_dataframe(self, level_stats: Dict) -> pd.DataFrame:
        """레벨별 통계 DataFrame 생성 - 재시도와 이탈 구분"""
        rows = []
        for level in range(1, len(self.game_levels) + 1):
            stats = level_stats[level]
            game_level = self.game_levels[level]
            
            # 재시도와 이탈 횟수 (이미 구분되어 있음)
            retry_count = stats['retries']
            abandon_count = stats['abandons']
            
            # 플레이 횟수 계산 (레벨을 시작한 유저 수)
            total_plays = len([s for s in stats.values() if isinstance(s, int) and s > 0])
            if total_plays == 0:
                total_plays = stats['successes'] + abandon_count  # 대안 계산
            
            success_rate = stats['successes'] / (stats['successes'] + abandon_count) if (stats['successes'] + abandon_count) > 0 else 0
            avg_attempts = stats['attempts'] / (stats['successes'] + abandon_count) if (stats['successes'] + abandon_count) > 0 else 0
            avg_retries = retry_count / (stats['successes'] + abandon_count) if (stats['successes'] + abandon_count) > 0 else 0
            avg_abandons = abandon_count / (stats['successes'] + abandon_count) if (stats['successes'] + abandon_count) > 0 else 0
            net_profit = stats['total_revenue'] - stats['total_cost']
            
            row = {
                'level': level,
                'n_pairs': game_level.n_pairs,
                'time_limit': game_level.time_limit,
                'success_reward': game_level.success_reward,
                'fail_reward': game_level.fail_reward,
                'turn_duration': game_level.turn_duration,
                'trick_cards': game_level.trick_cards,
                'total_plays': stats['successes'] + abandon_count,
                'attempts': stats['attempts'],
                'successes': stats['successes'],
                'failures': stats['failures'],
                'retry_count': retry_count,      # 재시도 횟수 (매출 발생)
                'abandon_count': abandon_count,  # 이탈 횟수 (매출 없음)
                'success_rate': success_rate,
                'avg_attempts_per_play': avg_attempts,
                'avg_retries_per_play': avg_retries,
                'avg_abandons_per_play': avg_abandons,
                'total_cost': stats['total_cost'],
                'total_revenue': stats['total_revenue'],
                'net_profit': net_profit,
                'profit_per_play': net_profit / (stats['successes'] + abandon_count) if (stats['successes'] + abandon_count) > 0 else 0
            }
            rows.append(row)
        
        return pd.DataFrame(rows)

def print_simulation_summary(user_df: pd.DataFrame, level_df: pd.DataFrame):
    """시뮬레이션 결과 요약 출력 - 유저 타입별 분석 추가"""
    print("\n" + "="*80)
    print("🎮 게임 시뮬레이션 결과 요약")
    print("="*80)
    
    total_users = len(user_df)
    total_revenue = user_df['total_revenue'].sum()
    total_cost = user_df['total_cost'].sum()
    net_profit = total_revenue - total_cost
    avg_levels = user_df['levels_completed'].mean()
    
    print(f"👥 전체 사용자: {total_users:,}명")
    print(f"💰 총 수익: {total_revenue:,.0f}원")
    print(f"💸 총 비용: {total_cost:,.0f}원")
    print(f"📊 순이익: {net_profit:,.0f}원")
    print(f"📈 사용자당 순이익: {net_profit/total_users:.2f}원")
    print(f"🎯 평균 완료 레벨: {avg_levels:.1f}단계")
    
    print(f"\n📈 레벨별 통계:")
    level_summary = level_df[['level', 'n_pairs', 'trick_cards', 'time_limit', 'success_rate', 
                             'avg_retries_per_play', 'total_cost', 'total_revenue', 'net_profit']].round(2)
    print(level_summary.to_string(index=False))
    
    print(f"\n🧠 메모리 크기별 성과:")
    memory_stats = user_df.groupby('memory_size').agg({
        'levels_completed': 'mean',
        'total_revenue': 'mean',
        'total_cost': 'mean',
        'net_profit': 'mean',
        'total_retries': 'mean'
    }).round(2)
    print(memory_stats)
    
    # 유저 타입별 분석 (realistic 모델인 경우에만)
    if 'user_type' in user_df.columns and user_df['user_type'].nunique() > 1:
        print(f"\n👤 유저 타입별 성과:")
        type_stats = user_df.groupby('user_type').agg({
            'levels_completed': 'mean',
            'total_revenue': 'mean', 
            'total_cost': 'mean',
            'net_profit': 'mean',
            'total_retries': 'mean',
            'user_id': 'count'
        }).round(2)
        type_stats.rename(columns={'user_id': 'user_count'}, inplace=True)
        print(type_stats)

def compare_retry_models(probability_df: pd.DataFrame, num_users: int = 5000) -> Dict:
    """Uniform vs Realistic 재시도 모델 비교"""
    print("🔄 재시도 모델 비교 분석 시작...")
    
    results = {}
    
    # 1. Uniform 모델 시뮬레이션
    print("\n1️⃣ Uniform 모델 시뮬레이션...")
    uniform_simulator = GameSimulator(
        probability_df=probability_df,
        retry_model='uniform'
    )
    uniform_user_df, uniform_level_df = uniform_simulator.run_simulation(num_users)
    results['uniform'] = {
        'user_df': uniform_user_df,
        'level_df': uniform_level_df,
        'simulator': uniform_simulator
    }
    
    # 2. Realistic 모델 시뮬레이션
    print("\n2️⃣ Realistic 모델 시뮬레이션...")
    realistic_simulator = GameSimulator(
        probability_df=probability_df,
        retry_model='realistic'
    )
    realistic_user_df, realistic_level_df = realistic_simulator.run_simulation(num_users)
    results['realistic'] = {
        'user_df': realistic_user_df,
        'level_df': realistic_level_df,
        'simulator': realistic_simulator
    }
    
    # 3. 비교 분석
    print("\n📊 모델 비교 결과")
    print("="*60)
    
    comparison_data = []
    
    for model_name, model_data in results.items():
        user_df = model_data['user_df']
        level_df = model_data['level_df']
        
        stats = {
            'Model': model_name.title(),
            'Total Revenue': user_df['total_revenue'].sum(),
            'Total Cost': user_df['total_cost'].sum(), 
            'Net Profit': user_df['net_profit'].sum(),
            'Avg Revenue/User': user_df['total_revenue'].mean(),
            'Avg Retries/User': user_df['total_retries'].mean(),
            'Avg Levels Completed': user_df['levels_completed'].mean(),
            'Revenue per Retry': user_df['total_revenue'].sum() / user_df['total_retries'].sum() if user_df['total_retries'].sum() > 0 else 0
        }
        comparison_data.append(stats)
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.round(2).to_string(index=False))
    
    # 4. 레벨별 상세 비교
    print(f"\n📈 레벨별 재시도율 비교:")
    level_comparison = pd.DataFrame({
        'Level': results['uniform']['level_df']['level'],
        'Uniform_Retry_Rate': results['uniform']['level_df']['avg_retries_per_play'],
        'Realistic_Retry_Rate': results['realistic']['level_df']['avg_retries_per_play'],
        'Uniform_Revenue': results['uniform']['level_df']['total_revenue'],
        'Realistic_Revenue': results['realistic']['level_df']['total_revenue']
    })
    level_comparison['Retry_Diff'] = level_comparison['Realistic_Retry_Rate'] - level_comparison['Uniform_Retry_Rate']
    level_comparison['Revenue_Diff'] = level_comparison['Realistic_Revenue'] - level_comparison['Uniform_Revenue']
    
    print(level_comparison.round(3).to_string(index=False))
    
    # 5. 유저 타입별 분석 (Realistic 모델만)
    if 'user_type' in realistic_user_df.columns:
        print(f"\n👤 유저 타입별 상세 분석 (Realistic 모델):")
        type_analysis = realistic_user_df.groupby('user_type').agg({
            'levels_completed': ['mean', 'std'],
            'total_revenue': ['mean', 'std'],
            'total_retries': ['mean', 'std'],
            'net_profit': ['mean', 'std'],
            'user_id': 'count'
        }).round(2)
        print(type_analysis)
    
    return results

# 사용 예시 및 테스트 함수
def demo_enhanced_simulator():
    """향상된 시뮬레이터 데모"""
    
    # 더미 확률 분포 데이터 생성 (실제 사용시에는 extract_probability_density_dataframe() 결과 사용)
    np.random.seed(42)
    
    # 메모리 크기별 턴 수 확률 분포 생성
    memory_sizes = range(1, 11)
    turn_columns = [f'turn_{i}' for i in range(2, 21)]  # turn_2부터 turn_20까지
    
    prob_data = []
    for memory in memory_sizes:
        # 메모리가 클수록 적은 턴 수에 높은 확률
        probs = np.random.dirichlet(np.linspace(10, 1, len(turn_columns)))
        row = dict(zip(turn_columns, probs))
        row['memory_size'] = memory
        prob_data.append(row)
    
    prob_df = pd.DataFrame(prob_data).set_index('memory_size')
    
    print("🚀 향상된 게임 시뮬레이터 데모 시작")
    print("="*50)
    
    # 1. Realistic 모델 단독 실행
    print("\n1️⃣ Realistic 모델 시뮬레이션")
    realistic_sim = GameSimulator(
        probability_df=prob_df,
        retry_model='realistic',
        max_retries=8
    )
    
    user_df, level_df = realistic_sim.run_simulation(num_users=2000)
    print_simulation_summary(user_df, level_df)
    
    # 2. 모델 비교
    print("\n" + "="*50)
    print("2️⃣ 모델 비교 분석")
    comparison_results = compare_retry_models(prob_df, num_users=2000)
    
    return comparison_results

# 실행
if __name__ == "__main__":
    demo_results = demo_enhanced_simulator()

🚀 향상된 게임 시뮬레이터 데모 시작

1️⃣ Realistic 모델 시뮬레이션
=== 2000명 유저 게임 시뮬레이션 시작 (재시도 모델: realistic) ===
진행률: 1000/2000 (50.0%)
진행률: 2000/2000 (100.0%)

📊 유저 타입 분포:
  regular: 891명 (44.5%)
  casual: 787명 (39.4%)
  hardcore: 322명 (16.1%)

🎮 게임 시뮬레이션 결과 요약
👥 전체 사용자: 2,000명
💰 총 수익: 1,134원
💸 총 비용: 6,696원
📊 순이익: -5,562원
📈 사용자당 순이익: -2.78원
🎯 평균 완료 레벨: 1.3단계

📈 레벨별 통계:
 level  n_pairs  trick_cards  time_limit  success_rate  avg_retries_per_play  total_cost  total_revenue  net_profit
     1        2            0         5.0          0.40                  0.33        1608            668        -940
     2        3            0         7.0          0.58                  0.30         930            242        -688
     3        6            0         9.0          0.71                  0.26         662            119        -543
     4        8            0        13.0          0.88                  0.14         580             47        -533
     5        8            2        15.0          0.85            

In [4]:
# 1. memory_v2에서 생성된 prob_df 사용 (이미 생성되어 있다고 가정)
# prob_df = extract_probability_density_dataframe(results)

# 2. 게임 시뮬레이터 생성
simulator = GameSimulator(prob_df)

# 3. 시뮬레이션 실행
user_df, level_df = simulator.run_simulation(num_users=22000)

# 4. 결과 출력
print_simulation_summary(user_df, level_df)

# 5. 트릭카드 영향 확인
print("\n" + "="*80)
print("트릭카드 영향 분석")
print("="*80)

# 레벨별 평균 소요 시간 확인
for level in [1, 2, 3, 4, 5, 6, 7]:
    game_level = simulator.game_levels[level]
    print(f"Level {level}: {game_level.n_pairs}쌍, 트릭카드 {game_level.trick_cards}개")
    print(f"  기본 턴 지속시간: {game_level.turn_duration:.2f}초")
    print(f"  효과적 턴 지속시간: {game_level.effective_turn_duration:.2f}초")
    print(f"  시간 증가율: {(game_level.effective_turn_duration / game_level.turn_duration - 1) * 100:.1f}%")
    print()

# 6. 결과 저장 (선택사항)
# user_df.to_csv('user_results.csv', index=False)
# level_df.to_csv('level_results.csv', index=False)

=== 22000명 유저 게임 시뮬레이션 시작 (재시도 모델: realistic) ===
진행률: 1000/22000 (4.5%)
진행률: 2000/22000 (9.1%)
진행률: 3000/22000 (13.6%)
진행률: 4000/22000 (18.2%)
진행률: 5000/22000 (22.7%)
진행률: 6000/22000 (27.3%)
진행률: 7000/22000 (31.8%)
진행률: 8000/22000 (36.4%)
진행률: 9000/22000 (40.9%)
진행률: 10000/22000 (45.5%)
진행률: 11000/22000 (50.0%)
진행률: 12000/22000 (54.5%)
진행률: 13000/22000 (59.1%)
진행률: 14000/22000 (63.6%)
진행률: 15000/22000 (68.2%)
진행률: 16000/22000 (72.7%)
진행률: 17000/22000 (77.3%)
진행률: 18000/22000 (81.8%)
진행률: 19000/22000 (86.4%)
진행률: 20000/22000 (90.9%)
진행률: 21000/22000 (95.5%)
진행률: 22000/22000 (100.0%)

📊 유저 타입 분포:
  casual: 8803명 (40.0%)
  regular: 9948명 (45.2%)
  hardcore: 3249명 (14.8%)

🎮 게임 시뮬레이션 결과 요약
👥 전체 사용자: 22,000명
💰 총 수익: 15,077원
💸 총 비용: 83,565원
📊 순이익: -68,488원
📈 사용자당 순이익: -3.11원
🎯 평균 완료 레벨: 1.9단계

📈 레벨별 통계:
 level  n_pairs  trick_cards  time_limit  success_rate  avg_retries_per_play  total_cost  total_revenue  net_profit
     1        2            0         5.0          0.90                  0.

In [14]:
level_df.total_cost.sum(), level_df.total_revenue.sum()

(83565, 15077)

In [17]:
user_df.total_cost.sum(), user_df.total_revenue.sum()

(83565, 15077)

### BEP 시뮬레이션

In [9]:
def find_break_even_point(simulator, prob_df, num_users=10000, max_iterations=20):
    """
    BEP를 찾기 위해 판당 보상을 조절하면서 시뮬레이션 실행
    
    Args:
        simulator: GameSimulator 인스턴스
        prob_df: 확률 분포 DataFrame
        num_users: 시뮬레이션 유저 수
        max_iterations: 최대 반복 횟수
    
    Returns:
        Dict: BEP 결과
    """
    print("=== BEP(Break-Even Point) 탐색 시작 ===")
    print("="*60)
    
    # 현재 보상 설정 저장
    original_rewards = {}
    for level, game_level in simulator.game_levels.items():
        original_rewards[level] = game_level.success_reward
    
    results = []
    
    # 낮은 판부터 보상 1씩 줄여가며 탐색
    for iteration in range(max_iterations):
        print(f"\n🔍 반복 {iteration + 1}: 보상 조정 중...")
        
        # 보상 조정 (낮은 판부터 1씩 감소)
        for level in range(1, len(simulator.game_levels) + 1):
            current_reward = original_rewards[level]
            adjusted_reward = max(1, current_reward - iteration)  # 최소 1원
            simulator.game_levels[level].success_reward = adjusted_reward
        
        # 현재 보상 설정 출력
        print("현재 보상 설정:")
        for level in range(1, len(simulator.game_levels) + 1):
            reward = simulator.game_levels[level].success_reward
            print(f"  Level {level}: {reward}원", end="  ")
        print()
        
        # 시뮬레이션 실행
        try:
            user_df, level_df = simulator.run_simulation(num_users=num_users)
            
            # 수익성 계산
            total_revenue = user_df['total_revenue'].sum()
            total_cost = user_df['total_cost'].sum()
            net_profit = total_revenue - total_cost
            profit_margin = (net_profit / total_revenue * 100) if total_revenue > 0 else 0
            
            # 결과 저장
            result = {
                'iteration': iteration + 1,
                'rewards': {level: simulator.game_levels[level].success_reward 
                           for level in range(1, len(simulator.game_levels) + 1)},
                'total_revenue': total_revenue,
                'total_cost': total_cost,
                'net_profit': net_profit,
                'profit_margin': profit_margin,
                'is_break_even': net_profit >= 0,
                'user_df': user_df,
                'level_df': level_df
            }
            results.append(result)
            
            print(f"💰 수익성: {total_revenue:,.0f}원 (수익) - {total_cost:,.0f}원 (비용) = {net_profit:,.0f}원")
            print(f"�� 마진: {profit_margin:.1f}%")
            
            # BEP 달성 확인
            if net_profit >= 0:
                print(f"🎯 BEP 달성! 반복 {iteration + 1}에서 손익분기점 도달")
                break
            else:
                print(f"❌ 아직 손실 상태 (손실: {abs(net_profit):,.0f}원)")
                
        except Exception as e:
            print(f"❌ 시뮬레이션 오류: {e}")
            break
    
    # 최종 결과 분석
    print("\n" + "="*60)
    print("�� BEP 탐색 결과 요약")
    print("="*60)
    
    if results:
        # BEP 달성한 결과 찾기
        bep_results = [r for r in results if r['is_break_even']]
        
        if bep_results:
            best_result = bep_results[0]  # 첫 번째 BEP 달성 결과
            print(f"✅ BEP 달성: 반복 {best_result['iteration']}")
            print(f"💰 최종 수익: {best_result['net_profit']:,.0f}원")
            print(f"�� 마진: {best_result['profit_margin']:.1f}%")
            print("\n🎯 최적 보상 설정:")
            for level, reward in best_result['rewards'].items():
                original = original_rewards[level]
                change = reward - original
                print(f"  Level {level}: {reward}원 (원래 {original}원, {change:+d}원)")
        else:
            print("❌ BEP 달성 실패: 최대 반복 횟수 내에서 손익분기점 도달 불가")
            last_result = results[-1]
            print(f"💰 최종 손실: {abs(last_result['net_profit']):,.0f}원")
            print(f"�� 마진: {last_result['profit_margin']:.1f}%")
    
    # 보상 원복
    for level, game_level in simulator.game_levels.items():
        game_level.success_reward = original_rewards[level]
    
    return results

def analyze_reward_sensitivity(results):
    """보상 민감도 분석"""
    print("\n" + "="*60)
    print("📊 보상 민감도 분석")
    print("="*60)
    
    if not results:
        print("분석할 결과가 없습니다.")
        return
    
    # 반복별 수익성 변화
    print("\n반복별 수익성 변화:")
    print("반복 | 수익(원) | 비용(원) | 순이익(원) | 마진(%)")
    print("-" * 50)
    
    for result in results:
        print(f"{result['iteration']:4d} | {result['total_revenue']:8,.0f} | {result['total_cost']:8,.0f} | "
              f"{result['net_profit']:9,.0f} | {result['profit_margin']:6.1f}")
    
    # 보상별 수익성 분석
    print("\n레벨별 보상 변화:")
    levels = list(range(1, len(results[0]['rewards']) + 1))
    print("반복 | " + " | ".join([f"L{i}" for i in levels]))
    print("-" * (6 + 4 * len(levels)))
    
    for result in results:
        rewards_str = " | ".join([f"{result['rewards'][i]:2d}" for i in levels])
        print(f"{result['iteration']:4d} | {rewards_str}")

# 사용 예시
def run_bep_analysis():
    """BEP 분석 실행"""
    # 1. 기본 시뮬레이터 생성
    simulator = GameSimulator(prob_df, retry_model='realistic')
    
    # 2. BEP 탐색 실행
    results = find_break_even_point(simulator, prob_df, num_users=5000, max_iterations=15)
    
    # 3. 민감도 분석
    analyze_reward_sensitivity(results)
    
    return results

# 실행
bep_results = run_bep_analysis()

=== BEP(Break-Even Point) 탐색 시작 ===

🔍 반복 1: 보상 조정 중...
현재 보상 설정:
  Level 1: 2원    Level 2: 2원    Level 3: 2원    Level 4: 2원    Level 5: 3원    Level 6: 3원    Level 7: 6원  
=== 5000명 유저 게임 시뮬레이션 시작 (재시도 모델: realistic) ===
진행률: 1000/5000 (20.0%)
진행률: 2000/5000 (40.0%)
진행률: 3000/5000 (60.0%)
진행률: 4000/5000 (80.0%)
진행률: 5000/5000 (100.0%)

📊 유저 타입 분포:
  regular: 2260명 (45.2%)
  casual: 1960명 (39.2%)
  hardcore: 780명 (15.6%)
💰 수익성: 3,600원 (수익) - 19,051원 (비용) = -15,451원
�� 마진: -429.2%
❌ 아직 손실 상태 (손실: 15,451원)

🔍 반복 2: 보상 조정 중...
현재 보상 설정:
  Level 1: 1원    Level 2: 1원    Level 3: 1원    Level 4: 1원    Level 5: 2원    Level 6: 2원    Level 7: 5원  
=== 5000명 유저 게임 시뮬레이션 시작 (재시도 모델: realistic) ===
진행률: 1000/5000 (20.0%)
진행률: 2000/5000 (40.0%)
진행률: 3000/5000 (60.0%)
진행률: 4000/5000 (80.0%)
진행률: 5000/5000 (100.0%)

📊 유저 타입 분포:
  regular: 2309명 (46.2%)
  hardcore: 761명 (15.2%)
  casual: 1930명 (38.6%)
💰 수익성: 2,662원 (수익) - 9,314원 (비용) = -6,652원
�� 마진: -249.9%
❌ 아직 손실 상태 (손실: 6,652원)

🔍 반복 3: 보상 조정 중...
현

In [31]:
# bep_results[1].keys()

import pandas as pd

# bep_results의 반복별 수익성 정보를 DataFrame으로 저장
pd.DataFrame([{
    '반복': result['iteration'],
    '수익(원)': result['total_revenue'],
    '비용(원)': result['total_cost'],
    '순이익(원)': result['net_profit'],
    '마진(%)': result['profit_margin']
} for result in bep_results])

,반복,수익(원),비용(원),순이익(원),마진(%)
0,1,3600,19051,-15451,-429.194444
1,2,2662,9314,-6652,-249.887303
2,3,2657,9293,-6636,-249.755363
3,4,2632,9248,-6616,-251.367781
4,5,2501,9253,-6752,-269.972011
5,6,2599,9197,-6598,-253.866872
6,7,2608,9103,-6495,-249.041411
7,8,2550,9297,-6747,-264.588235
8,9,2575,9305,-6730,-261.359223
9,10,2593,9251,-6658,-256.768222


In [47]:
key = list(range(1, len(bep_results[0]['rewards']) + 1))
pd.DataFrame([bep_results[i-1]['rewards'] for i in key])

,1,2,3,4,5,6,7
0,2,2,2,2,3,3,6
1,1,1,1,1,2,2,5
2,1,1,1,1,1,1,4
3,1,1,1,1,1,1,3
4,1,1,1,1,1,1,2
5,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1


In [38]:
list(range(1, len(bep_results[0]['rewards']) + 1))

[1, 2, 3, 4, 5, 6, 7]